In [1]:
import warnings 

In [2]:
import numpy as np
import cv2
import mediapipe as mp
from tensorflow.keras.models import load_model
import torch
import h5py
from sklearn.preprocessing import LabelEncoder 
import numpy as np
from sklearn.preprocessing import LabelEncoder 
import numpy as np
warnings.filterwarnings('ignore', category=UserWarning)

In [3]:

def load_data(h5_filepath):
    with h5py.File(h5_filepath, 'r') as hf:
        Y = np.array([cadena.decode('utf-8')for cadena in hf['Y'][:]])
    return Y

Y=load_data("datos.h5")

In [4]:
label_encoder = LabelEncoder()
label_encoder.fit(Y)

LabelEncoder()

In [5]:

model = load_model('modelo.keras')


In [6]:
def calcular_bounding_box(x_min,x_max,y_min,y_max, width, height, margen=0.1):
    box_width = x_max - x_min
    box_height = y_max - y_min
    x_min = max(0, x_min - margen * box_width)
    x_max = min(width, x_max + margen * box_width)
    y_min = max(0, y_min - margen * box_height)
    y_max = min(height, y_max + margen * box_height)
    
    return int(x_min), int(x_max), int(y_min), int(y_max)

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import time

modelo_landmarks = mp.solutions.drawing_utils
modelo_manos = mp.solutions.hands

video = cv2.VideoCapture(0)#"./videos/CERDO_STA.CRUZ.mp4"

# List to store exactly 5 recognized signs
recognized_signs = []
focus_duration = 5  # Time to focus on each character in seconds
start_time = time.time()

# Initialize hand detection model
with modelo_manos.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7
) as hands:
    while len(recognized_signs) < 5:
        r, image = video.read()
        if r is None:
            break

        image = cv2.resize(image, (640, 480))
        height, width, _ = image.shape
        image = cv2.flip(image, 1)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Detect hands
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks is not None:
            for marcador_manos in results.multi_hand_landmarks:
                landmarks = []
                X = []
                x_min, x_max = 999, -999
                y_min, y_max = 999, -999

                # Collect hand landmarks and calculate bounding box
                for punto in marcador_manos.landmark:
                    landmarks += [punto.x, punto.y]
                    x_min = min(punto.x, x_min)
                    x_max = max(punto.x, x_max)
                    y_min = min(punto.y, y_min)
                    y_max = max(punto.y, y_max)

                X.append(landmarks)
                X = np.array(X)
                
                # Calculate box coordinates in pixels
                x_min, x_max, y_min, y_max = (
                    int(x_min * width), int(x_max * width),
                    int(y_min * height), int(y_max * height)
                )

                # Make prediction
                pred = np.argmax(model.predict(X))
                label = label_encoder.inverse_transform([pred])[0]

                # Check if 5 seconds have passed to save the character
                if time.time() - start_time >= focus_duration:
                    recognized_signs.append(label)  # Add label to recognized signs
                    print(f"Recognized signs so far: {recognized_signs}")
                    start_time = time.time()  # Reset timer for next character

                    # Stop once we have 5 characters
                    if len(recognized_signs) == 3:
                        print(f"Final recognized signs: {recognized_signs}")
                        break

                # Draw bounding box and label
                label_offset = 19
                label_x = x_min + (x_max - x_min) // 2
                label_y = y_max + label_offset
                cv2.putText(image, label, (label_x, label_y), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

                # Draw landmarks and connections
                modelo_landmarks.draw_landmarks(
                    image, marcador_manos, modelo_manos.HAND_CONNECTIONS,
                    modelo_landmarks.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=4),
                    modelo_landmarks.DrawingSpec(color=(0, 255, 0), thickness=2)
                )

        cv2.imshow("Hand Recognition", image)

        # Press 'q' to exit early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

video.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━

In [8]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# from accelerate import init_empty_weights

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    # device_map="auto",
    # torch_dtype=torch.bfloat16,
).to('cpu')

c:\Users\laime\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:24<00:00, 12.45s/it]


In [9]:
def genFrase(listWords):
    input_text = f"""
Question: [H,O,L,A,_,P,A,T,O]
Answer: Hola Pato

Question: [S,O,Y,_,D,E,_,B,O,L,I,V,I,A]
Answer: Soy de Bolivia

Question: [S,O,Y,_,R,E,L,O,J]
Answer: Soy reloj

Question: [1,2,3]
Answer: Uno, dos, tres

Question: {listWords}
Answer:"""

    model.to("cpu")
    input_ids = tokenizer(input_text, return_tensors="pt").to("cpu")

    outputs = model.generate(
        **input_ids, max_new_tokens=10, temperature=0.7, top_p=0.9, top_k=50
    )
    ans = tokenizer.decode(outputs[0])
    
    if "Answer:" in ans:
        generated_text = ans.split("Answer:")[-1].strip()
    
    generated_text = generated_text.split("\n")[0].strip()  # Handle extra newlines
    
    return generated_text


In [ ]:
genFrase("[H,O,L,A,_,P,A,T,O]")

'Hola Pato'

In [19]:
genFrase(['D,E,E,P,_,L,E,A,R,N,I,N,G'])

'Deep learning'

In [24]:
genFrase(['H', 'D', 'O', 'L', 'A'])

'Hola'

In [25]:
genFrase("[H,O,D,L,A]")

'Hola'

In [18]:
genFrase(['H', 'O', 'L', 'A', 'BOLIVIA'])

'Hola Bolivia'